This notebook has all the methods that permits to manage an image and to create an artificial LiDAR instance

In [1]:
from skimage import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
map_img = io.imread("data/my_map.png",)

In [3]:
#function of the creation of the instace LiDAR

def create_lidar(x, y, r, map):
    #correzione assi
    #x=map.shape[0]-x-1
    #r=r-180

    #if the position of the image is not a free space (obstacle or unknown) it returns None
    if map[x][y] != 254:
        return None
    else:
        #create an artificial LiDAR based on the neibourhood pixels
        result = look_around(map,x,y,r,int(20/0.05))
        return result

def look_around(map,x,y,r,dist):
    result = {}

    for i in range(720):
        angle = ((i + r) % 720)
        lidar_range = get_range(map, x, y, np.radians(angle/2),dist)
        result["range{}".format(i)] = lidar_range*0.05
        result["angle{}".format(i)] = np.around(np.radians((i-360)/2),decimals=5)  

    return result

def get_range(map,x,y,rad,dist):

    for i in range(dist):
        x_range = np.cos(rad) * i
        y_range = np.sin(rad) * i
        if map[x+int(x_range)][y+int(y_range)] != 254:
            break

    if map[x+int(x_range)][y+int(y_range)] == 0:
        
        return min(np.sqrt((np.power(int(x_range),2))+(np.power(int(y_range),2))), dist)
    
    return dist

In [4]:
instances= []

width = map_img.shape[0]
heigh = map_img.shape[1]

#random position/rotation generator

In [ ]:
#multi rot position

tot_inst= 2350

while len(instances) < tot_inst:
    i = np.random.randint(0,width)
    j = np.random.randint(0,heigh)
    rep = np.random.randint(1,15)
    for c in range(rep):
        r = np.random.randint(-360,360)
        result = create_lidar(i,j,r,map_img)
        if result is not None:
            instances.append([result,i*0.05,j*0.05,np.radians(r/2)])
            print(len(instances))

#get path from file

In [ ]:
with open("data/laser_log2303.csv", "r") as data:
    laser_db = pd.read_csv(data, delimiter=";").copy()

In [15]:
trail = []
for i in laser_db.values:
    if i[2] == -3.14159:
        trail.append(i[4:7])

In [ ]:
instances = []

#for n in range(3):
for i in range(120):
    for j in range(4):
        x = i+3
        y = j+60
        r = 0#np.random.randint(-360,360)
        laser_art = create_lidar(x,y,r,map_img)
        instances.append([laser_art,x*0.05,y*0.05,np.radians(r/2)])
        print(len(instances))

In [ ]:
instances = []
c=0

for i in trail:
    x = int(i[1]//0.05)
    y = int(i[0]//0.05)
    laser_art = create_lidar(x,y,((np.degrees(i[2])*2)),map_img)
    instances.append([laser_art,i[1],i[0],i[2]])
    print(c)
    c+=1


#save instances

In [6]:
with open('datasetmymap_multirot.csv', 'w') as f:
    f.write("cnt;time;angle;range;pos_x;pos_y;pos_yaw\n")
    c=1.0
    for item in instances:
        
        for i in range(720):
            if item[0] is not None:
                f.write("{};0;{};{};{};{};{}\n".format(c,item[0]["angle{}".format(i)],item[0]["range{}".format(i)],item[2],item[1],item[3]))
        
        c += 1

Refactor data

In [5]:
import pandas as pd

with open("data/datasetmymap(3-123,60-64)_unidir.csv", "r") as data:
    laser_db = pd.read_csv(data, delimiter=";").copy()

laser_db.loc[:,"pos_x"] = laser_db.loc[:,"pos_x"] * 0.05
laser_db.loc[:,"pos_y"] = laser_db.loc[:,"pos_y"] * 0.05

In [6]:
with open('datasetmymap(3-123,60-64)_unidir.csv', 'w') as f:
    f.write("cnt;time;angle;range;pos_x;pos_y;pos_yaw\n")
    for item in laser_db.values:
        
        f.write("{};{};{};{};{};{};{}\n".format(item[0],item[1],item[2],item[3],item[4],item[5],item[6]))